In [6]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::when()       masks foreach::when()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-4

Warning message:
“package ‘lme4’ was built under R version 4.2.3”
Warning mess

In [7]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,10,20,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [8]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

In [9]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()

corpus_list<-c("coha")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("UnTagged")

ppmi_setting_list<-c("RAW")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [10]:
for (c in corpus_list){
  for (t in tagged_list) {
    for (f in feature_setting_list) {
      for (p in ppmi_setting_list){
        for (a in comp_setting_list){
          for (i in time_off) {
            
            for (j in cut_off) {
              
              
              print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
              
              na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
              nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
              
              for (r in ratings_list){
                na_df<-na_df %>% filter(source==r)
                nona_df<-nona_df %>% filter(source==r)
                
                trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_na<-na_df %>% select(compositionality)
                trainY_na<-trainY_na$compositionality
                
                
                trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                trainY_nona<-nona_df %>% select(compositionality)
                trainY_nona<-trainY_nona$compositionality
                
                if (dim(trainX_nona)[1]<10) {
                  break
                }
                for (s in seed_list)  {
                  set.seed(s)
                  seeds <- vector(mode = "list", length = 14)
                  for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                  #for the last model
                  seeds[[14]]<-sample.int(1000, 1)
                  
                  ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                      preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                    trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                    preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                  )
                  
                  lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                               preProc = c("nzv","center", "scale")
                  )
                  
                  
                  ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                             trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                             preProcess = c("nzv","medianImpute", "center", "scale")
                  )
                  
                  
                  perf_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                  perf_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                  perf_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                  
                  
                  perf_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                  perf_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                  perf_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                  
                  varimp_elastic_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_na)$importance))
                  varimp_ridge_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_na)$importance))
                  varimp_lasso_na<-data.frame(feature=f,impute="r",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_na)$importance))
                  
                  varimp_elastic_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_nona)$importance))
                  varimp_ridge_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_nona)$importance))
                  varimp_lasso_nona<-data.frame(feature=f,impute="python",tag=t,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_nona)$importance))    
                  
                  
                  
                  list_of_rsqr[[m]]<-perf_elastic_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_elastic_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_na
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_ridge_nona
                  m<-m+1
                  
                  list_of_rsqr[[m]]<-perf_lasso_na
                  m<-m+1 
                  
                  list_of_rsqr[[m]]<-perf_lasso_nona
                  m<-m+1 
                  
                  if (i==10000) {
                    list_of_vi_10000[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10000[[v]]<-varimp_lasso_nona
                    v<-v+1
                    
                  }
                  else if (i==10) {
                    list_of_vi_10[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_10[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                  else if (i==20) {
                    list_of_vi_20[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_20[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==50) {
                    list_of_vi_50[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_50[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  else if (i==100) {
                    list_of_vi_100[[v]]<-varimp_elastic_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_elastic_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_ridge_nona
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_na
                    v<-v+1
                    list_of_vi_100[[v]]<-varimp_lasso_nona
                    v<-v+1
                  }
                  
                }
                
              }
            }
          }
        }
      }
    }
  }
}

[1] "coha RAW Aware withSetting UnTagged 10000 0"
[1] "coha RAW Aware withSetting UnTagged 10000 10"
[1] "coha RAW Aware withSetting UnTagged 10000 50"
[1] "coha RAW Aware withSetting UnTagged 10000 100"
[1] "coha RAW Aware withSetting UnTagged 10000 500"
[1] "coha RAW Aware withSetting UnTagged 10000 1000"
[1] "coha RAW Aware withSetting UnTagged 10 0"
[1] "coha RAW Aware withSetting UnTagged 10 10"
[1] "coha RAW Aware withSetting UnTagged 10 50"
[1] "coha RAW Aware withSetting UnTagged 10 100"
[1] "coha RAW Aware withSetting UnTagged 10 500"
[1] "coha RAW Aware withSetting UnTagged 10 1000"
[1] "coha RAW Aware withSetting UnTagged 20 0"
[1] "coha RAW Aware withSetting UnTagged 20 10"
[1] "coha RAW Aware withSetting UnTagged 20 50"
[1] "coha RAW Aware withSetting UnTagged 20 100"
[1] "coha RAW Aware withSetting UnTagged 20 500"
[1] "coha RAW Aware withSetting UnTagged 20 1000"
[1] "coha RAW Aware withSetting UnTagged 50 0"
[1] "coha RAW Aware withSetting UnTagged 50 10"
[1] "coha RAW 

In [11]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
statistics<-rsquared_df %>% group_by(impute,setting,dataset,ml_algo,timespan,cutoff) %>% 
summarise(n=mean(n),meanRsquared=mean(TrainRsquared),sdRsquared=sd(TrainRsquared),meanCorr=mean(TrainSpearman),sdCorr=sd(TrainSpearman))

statistics %>% arrange(desc(meanCorr)) %>% head()

`summarise()` has grouped output by 'impute', 'setting', 'dataset', 'ml_algo',
'timespan'. You can override using the `.groups` argument.


impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
r,Aware,reddy,elastic,100,0,91,0.5417695,0.01690080,0.7111842,0.02512862
r,Aware,reddy,elastic,50,0,91,0.5130545,0.02881806,0.7056447,0.02410769
python,Agnostic,reddy,elastic,50,0,91,0.5257513,0.02076735,0.7044751,0.02388376
r,Agnostic,reddy,elastic,100,0,91,0.5247243,0.02851501,0.7021364,0.03073532
python,Aware,reddy,elastic,50,0,91,0.5289459,0.02487694,0.7011875,0.02114813
r,Agnostic,reddy,elastic,50,0,91,0.5183288,0.02760955,0.7002314,0.03412555


In [16]:
varimp_df<-bind_rows(list_of_vi)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0

In [6]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)

In [16]:
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
601,woSetting,r,UnTagged,Aware,reddy,coha,elastic,90,1001,20,50,RAW,glmnet,0.4243327,0.8095238
603,woSetting,r,UnTagged,Aware,reddy,coha,rigde,90,1001,20,50,RAW,glmnet,0.4243327,0.8095238
535,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1991,20,0,RAW,glmnet,0.5087794,0.7666667
537,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1991,20,0,RAW,glmnet,0.4787298,0.7666667
494,woSetting,python,UnTagged,Aware,reddy,coha,elastic,91,1221,20,0,RAW,glmnet,0.4910019,0.7233765
498,woSetting,python,UnTagged,Aware,reddy,coha,lasso,91,1221,20,0,RAW,glmnet,0.4860564,0.7229652
511,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,20,0,RAW,glmnet,0.5242040,0.7142857
513,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1551,20,0,RAW,glmnet,0.5091473,0.7142857
271,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,10,0,RAW,glmnet,0.5784342,0.7123575


In [17]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
337,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,10,10,RAW,glmnet,0.7570484,0.5466651
577,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,20,10,RAW,glmnet,0.5963514,0.5952567
271,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1551,10,0,RAW,glmnet,0.5784342,0.7123575
273,woSetting,r,UnTagged,Aware,reddy,coha,rigde,91,1551,10,0,RAW,glmnet,0.5784342,0.7123575
553,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1221,20,10,RAW,glmnet,0.5552072,0.6696131
557,woSetting,r,UnTagged,Aware,reddy,coha,lasso,91,1221,20,10,RAW,glmnet,0.5501744,0.5785527
584,woSetting,python,UnTagged,Aware,reddy,coha,elastic,91,1771,20,10,RAW,glmnet,0.5478973,0.7104329
517,woSetting,r,UnTagged,Aware,reddy,coha,elastic,91,1661,20,0,RAW,glmnet,0.5441187,0.6338383
521,woSetting,r,UnTagged,Aware,reddy,coha,lasso,91,1661,20,0,RAW,glmnet,0.5378063,0.6168681


In [18]:
write.csv(rsquared_df,"rsquared_Tagged_PPMI_Agnostic.csv",row.names = FALSE)

In [19]:
write.csv(varimp_df,"varimp_Tagged_PPMI_Agnostic.csv",row.names = FALSE)